In [1]:
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
access_token = ""
repo_owner = "octokit"
repo_name = "graphql.js"
target_label= ["bug"]

In [3]:
client = Client(
    transport=RequestsHTTPTransport(
      url = "https://api.github.com/graphql",
      use_json = True,
      headers = {
        "Content-type": "application/json",
        "Authorization": "Bearer {}".format(access_token)
      },
      retries = 3,
    ),
    fetch_schema_from_transport=True,
)
resp = client.execute(
  gql("""query($owner:String!, $name:String!, $label:[String!]) {
    repository(owner: $owner, name: $name) {
      pullRequests(labels: $label, last: 100, states: [MERGED, OPEN]) {
        edges {
          node {
            number
            title
            url
            author {
              login
            }
            createdAt
            mergedAt
            comments {
              totalCount
            }
            additions
            deletions
            commits(first: 1) {
              totalCount
              nodes {
                commit {
                  authoredDate
                }
              }
            }
            reviews(first: 1) {
              totalCount
              nodes {
                ... on PullRequestReview {
                  createdAt
                  author {
                    login
                  }
                }
              }
            }
          }
        }
      }
    }
  }"""),
  variable_values={
    "owner": repo_owner,
    "name": repo_name,
    "label": target_label
  }
)

In [4]:
data = resp['repository']['pullRequests']['edges']

In [5]:
columns = [
    "number",
    "title",
    "url",
    "author",
    "createdAt",
    "mergedAt",
    "additions",
    "deletions",
    "totalCommits",
    "firstCommittedAt",
    "reviewer",
    "firstReviewedAt",
    "totalReviewComments",
]
df = pd.DataFrame(index=range(len(data)), columns=columns)

In [6]:
for i, value in enumerate(data):
    pr = value["node"]
    df.at[i, "number"] = pr["number"]
    df.at[i, "title"] = pr["title"]
    df.at[i, "url"] = pr["url"]
    df.at[i, "author"] = pr["author"]["login"]
    df.at[i, "createdAt"] = pr["createdAt"]
    df.at[i, "mergedAt"] = pr["mergedAt"]
    df.at[i, "additions"] = pr["additions"]
    df.at[i, "deletions"] = pr["deletions"]
    df.at[i, "totalCommits"] = pr["commits"]["totalCount"]
    df.at[i, "firstCommittedAt"] = pr["commits"]["nodes"][0]["commit"]["authoredDate"]
    df.at[i, "reviewer"] = pr["reviews"]["nodes"][0]["author"]["login"] if len(pr["reviews"]["nodes"]) > 0 else None
    df.at[i, "firstReviewedAt"] = pr["reviews"]["nodes"][0]["createdAt"] if len(pr["reviews"]["nodes"]) > 0 else None
    df.at[i, "totalReviewComments"] = pr["reviews"]["totalCount"]

In [7]:
df

,number,title,url,author,createdAt,mergedAt,additions,deletions,totalCommits,firstCommittedAt,reviewer,firstReviewedAt,totalReviewComments
0,168,fix: syntax error in the code example,https://github.com/octokit/graphql.js/pull/168,lukyth,2020-08-12T07:05:03Z,2020-08-14T15:55:08Z,2,3,1,2020-08-12T07:04:42Z,gr2m,2020-08-12T16:17:13Z,3
1,186,fix: handle `baseUrl` suffix for GHES,https://github.com/octokit/graphql.js/pull/186,gr2m,2020-09-09T17:31:56Z,2020-09-09T18:25:29Z,35,0,4,2020-09-09T17:33:29Z,lencioni,2020-09-09T18:00:24Z,4
2,187,fix: throw helpful error when `query` is used ...,https://github.com/octokit/graphql.js/pull/187,gr2m,2020-09-10T20:22:30Z,2020-09-10T20:31:09Z,49,19,2,2020-09-10T20:22:00Z,None,None,0
3,263,Update browser import,https://github.com/octokit/graphql.js/pull/263,domnantas,2021-03-08T18:22:57Z,2021-03-08T19:32:50Z,1,1,1,2021-03-08T18:22:21Z,gr2m,2021-03-08T19:32:15Z,1
4,265,"fix: throw helpful error in case ""query"", ""url...",https://github.com/octokit/graphql.js/pull/265,gr2m,2021-03-09T23:26:50Z,2021-03-09T23:32:28Z,56,4,2,2021-03-09T23:26:36Z,None,None,0
5,281,fix (README): Corrected package import,https://github.com/octokit/graphql.js/pull/281,loke-dev,2021-05-17T20:36:55Z,2021-05-17T23:59:18Z,1,1,1,2021-05-17T20:35:55Z,gr2m,2021-05-17T23:56:37Z,1
6,314,feat(typescript): expose `type` property for e...,https://github.com/octokit/graphql.js/pull/314,devversion,2021-08-30T18:02:10Z,2021-08-31T17:38:21Z,2,0,2,2021-08-30T17:59:52Z,gr2m,2021-08-31T17:26:31Z,3
7,335,Fix `createAppAuth` arguments `id` to `appId` ...,https://github.com/octokit/graphql.js/pull/335,KeisukeYamashita,2022-02-20T05:42:30Z,None,1,1,1,2022-02-20T05:40:59Z,None,None,0


In [8]:
df.to_csv("pr_data.csv", index=False)